This Code calucates the tidal evolution for a spectrum of particle sizes and initial locations to produce Fig 3a in the paper. 


In [5]:
%pylab

from astropy import constants
from scipy import integrate

G_val = constants.G.si.value # Gravitational Constant

M_mars = 6.41693*(10**23.) # Mars mass, kg
R_mars = 3386.*1e3         # Mars radius, m
rho_mars = 3934            # kg/m^3
V_phobos = 5689*1e9        # Phobos volume, +/- 60*1e9 m^3
M_phobos = 1.065*(10**16.) # Phobos Mass, +/- .015 kg
R_phobos = 11.1*1e3        # Phobos Radius , m
rho_phobos = 1873.         # +/- 31 kg/m^3
ecc_phobos =  0.01511      # initial eccentricity
a_phobos = 9375*1e3        # Semi-major Axis, m 
w_phobos = 1./27562.       # 1/Rotation period of phobos ~ 7hr 39 mins, 1/s

k2_mars = .148  # In Nimmo 2013; .148 +/- .017 ; Other Sets in Jacobsen 2014 - 0.1837 +/- .0009
Q_mars = 88.    # In Nimmo 2013 88. +/- 16, 170+/- 20 ; Other Sets in Jacobsen 2014 - 99.57 +/- 4.9 

mu_phobos_monolith = 8e4 # Dimensionless, from 8e4 - 8e6
mu_phobos_rubble = sqrt(mu_phobos_monolith/1e-2) # Dimensionless, ~ sqrt(mu/epsilon_yield_strain); ~ from 2.85e3 - 2.85e4
k2_phobos = 1.5/(1. + mu_phobos_rubble)
Q_phobos = 100. # Can be between 50 - 100

mu_phobos_C1 = (4.*pi/19.)*(rho_phobos**2.)*(R_phobos**2.)*G_val/k2_phobos
const_Ecc1 = -(57./8.)*(k2_mars/Q_mars)*sqrt(G_val/M_mars)*(R_mars**5.)*M_phobos
const_Ecc2 = -(21./2.)*(k2_phobos/Q_phobos)*sqrt(G_val*M_mars)*(R_phobos**5.)*(M_mars/M_phobos)

kappa = (rho_phobos/rho_mars)*(R_phobos/R_mars)**3.
rad_ratio = R_phobos/R_mars
    
mu_mars_C1 = (4.*pi/19.)*(rho_mars**2.)*(R_mars**2.)*G_val/k2_mars
const_a1 = (8.*sqrt(3)/19.)*(pi*G_val)**1.5*(R_mars**2.)*(rho_mars**2.5)*kappa*sqrt(1.+kappa)*(R_mars**5.5)/mu_mars_C1/Q_mars
    
def dX_dt(t,X):
    """ X is [a,e,w_s,E] - for moon/Phobos , E is the Energy, a is in units of R_mars """
    dt_e = (const_Ecc1 + const_Ecc2)*(X[1])/((X[0]*R_mars)**6.5)
    w_t1 = + (19./22.)*(rad_ratio**2.)/(X[0]**2.)
    w_t2 = + (380./459.)*(rad_ratio**4.)/(X[0]**4.)
    w_t3 = + (475./584.)*(rad_ratio**6.)/(X[0]**6.)
    w_t4 = + (133./165.)*(rad_ratio**8.)/(X[0]**8.)
    a_t1 = + (19./22.)/(X[0]**2.)
    a_t2 = + (380./459.)/(X[0]**4.)
    a_t3 = + (475./584.)/(X[0]**6.)
    a_t4 = + (133./165.)/(X[0]**8.)
    dt_a = -1.*(const_a1/((X[0]*R_mars)**5.5))*(1. + 51.*X[1]*X[1]/4.)*(1. + a_t1 + a_t2 + a_t3 + a_t4) # Is really dt_a/R_mars
    if (X[1] <=0.):
        dt_e =0.
    if (X[0] <=1.):
        dt_a =0.
        dt_e =0.
        dt_w =0.
        dt_E =0.
    return array([dt_a,dt_e])

max_t =80.*1e6*3.154e7                            # unit is per seconds
dt = 1e4*3.154e7
X_init = array([a_phobos/R_mars,ecc_phobos])               # initials conditions: [a,e]
abc = integrate.ode(dX_dt).set_integrator('vode',nsteps=1e8)
abc.set_initial_value(X_init,0.0)

t_vX=[]
a_intX=[]
e_intX=[]

a_intX.append(X_init[0])
e_intX.append(X_init[1])
t_vX.append(0.)
while abc.t < max_t :
	#print abc.t/max_t
	X = abc.integrate(abc.t+dt)
	a_intX.append(X[0])
	e_intX.append(X[1])
	t_vX.append(abc.t)
	#print abc.t/max_t
    
a_int = array(a_intX)
e_int = array(e_intX)
t_v = array(t_vX)

r_peri =a_int*(1. - e_int) # periapse location 
r_apo = a_int*(1. + e_int) # apoapse location 
Vel_r_max = sqrt(G_val*M_mars*(2./r_apo- 1./a_int)/R_mars)# Total Velocity at the periapse for the given a, m/s
Vel_r_min = sqrt(G_val*M_mars*(2./r_peri - 1./a_int)/R_mars)# Total Velocity at the periapse for the given a, m/s

a_int_orig = a_int.copy()
e_int_orig = e_int.copy()


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
# Here we calculate the timescale for tidal orbital evolution for 
# a given particle size and at a given Roche (i.e initial orbital location)
# to the top of Mars's atmosphere

a_roche_list = linspace(a_int_orig[0],1.+5.*160./R_mars,25)
# Specify orbital eccentricity of the particles to be equal to the eccentricity of Phobos at that orbital location
e_roche_list = np.interp(a_roche_list, sort(a_int_orig), sort(e_int_orig)) 

mu_phobos_monolith = 8e5 # Dimensionless, from 8e4 - 8e6
mu_phobos_rubble = sqrt(mu_phobos_monolith/1e-2) # Dimensionless, ~ sqrt(mu/epsilon_yield_strain); ~ from 2.85e3 - 2.85e4
k2_phobos = 1.5/(1. + mu_phobos_rubble)
Q_phobos = 100. # Can be between 50 - 100

# Range of particle sizes (in m)
size_range = array([5,15,25,50,75,100,250,350,500,600,750,1000,1500,2000,2500,3000,3500,4000,4500,5000,6000]) 

time_scales_a = zeros(525)
time_scales_R = zeros(525)
time_scales_t = zeros(525)
time_scales_mark = zeros(525)
time_scales_mark[502] =1
time_scales_mark[503] =1
time_scales_mark[513] =1
time_scales_mark[514] =1
time_scales_mark[514:525] =1

for i in range (0,a_roche_list.shape[-1]) :
    a_phobos = a_roche_list[i]*R_mars               # semi-major axis of the particle
    ecc_phobos = e_roche_list[i]                    # eccentricity of the particle
    w_phobos = sqrt(G_val*M_mars/(a_phobos)**3.)/2./pi  # spin-period of the particle
    for k in range(0,size_range.shape[-1]):
        R_phobos = size_range[k]                        # size of the particle , m
        M_phobos = (4.*pi/3.)*rho_phobos*(R_phobos)**3. # Mass of the particle, same density as Phobos 
        
        mu_phobos_C1 = (4.*pi/19.)*(rho_phobos**2.)*(R_phobos**2.)*G_val/k2_phobos
        const_Ecc1 = -(57./8.)*(k2_mars/Q_mars)*sqrt(G_val/M_mars)*(R_mars**5.)*M_phobos
        const_Ecc2 = -(21./2.)*(k2_phobos/Q_phobos)*sqrt(G_val*M_mars)*(R_phobos**5.)*(M_mars/M_phobos)
        
        kappa = (rho_phobos/rho_mars)*(R_phobos/R_mars)**3.
        rad_ratio = R_phobos/R_mars
        
        mu_mars_C1 = (4.*pi/19.)*(rho_mars**2.)*(R_mars**2.)*G_val/k2_mars
        const_a1 = (8.*sqrt(3)/19.)*(pi*G_val)**1.5*(R_mars**2.)*(rho_mars**2.5)*kappa*sqrt(1.+kappa)*(R_mars**5.5)/mu_mars_C1/Q_mars
    
        max_t =10e9*3.154e7                            # unit is per seconds
        dt = 1e5*3.154e7
        # This is just to change the total integration time for a few particle sizes
        if (time_scales_mark[i*21+k] >0):                             
            max_t =1e6*3.154e7                                       # unit is per seconds
            dt = 10.*3.154e7
        X_init = array([a_phobos/R_mars,ecc_phobos])               # initials conditions: [a,e]
        abc = integrate.ode(dX_dt).set_integrator('vode',nsteps=1e8)
        abc.set_initial_value(X_init,0.0)

        t_vX=[]
        a_intX=[]
        e_intX=[]

        a_intX.append(X_init[0])
        e_intX.append(X_init[1])
        t_vX.append(0.)
        while abc.t < max_t :
            #print abc.t/max_t
            X = abc.integrate(abc.t+dt)
            a_intX.append(X[0])
            e_intX.append(X[1])
            t_vX.append(abc.t)
            #print abc.t/max_t
    
        a_int = array(a_intX)
        t_v = array(t_vX)
        time_scales_a[i*21+k] = a_phobos  # Save particle's initial location
        time_scales_R[i*21+k] = R_phobos  # Save particle's initial size
        time_scales_t[i*21+k] = t_v[a_int>1.+160./R_mars][-1]/1e7/1e6 # in Myr, time when particle hits Martian atmosphere
        if (t_v[a_int>1.+160./R_mars][-1]<=0.):
            print (t_v[a_int>1.+160./R_mars][-1],a_phobos/R_mars,R_phobos,i*21+k)

In [6]:
# Plotting the timescales to make the contour plot (Fig 3a)

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LogNorm
import matplotlib.gridspec as gridspec

fig_width_pt = 2.*130.0  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inch
golden_mean = (sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height = 1.3*fig_width*golden_mean      # height in inches
params = {'axes.labelsize': 10,
          'text.fontsize': 8,
          'text.fontweight': 30,
          'legend.fontsize': 6,
          'xtick.labelsize': 8.,
          'axes.linewidth': .3,
          'ytick.labelsize': 8.}
pylab.rcParams.update(params)

fig, axes = plt.subplots(nrows=1, ncols=1, sharex=True, sharey=True,figsize=(fig_width,fig_height),dpi=250)
gs = gridspec.GridSpec(1, 2,width_ratios=[15,1])


# the 1st subplot
ax1 = plt.subplot(gs[0])
lvls = np.logspace(-3,4.1,200)
Z1 = time_scales_t.reshape(25,21)/3.154
CF = ax1.contourf(time_scales_R.reshape(25,21)/1000.,time_scales_a.reshape(25,21)/R_mars,Z1,
                  norm = LogNorm(),
                  levels = lvls,cmap='YlOrRd'
                 )
ax1.set_ylabel('a$_{disruption}$ / R$_{Mars}$',fontsize=10,labelpad=5)
ax1.set_xlabel('$Particle \, Size\, (km)$',fontsize=10,labelpad=4)
plt.gcf().subplots_adjust(bottom=0.15)
plt.gcf().subplots_adjust(left=.2)
ax1.set_xscale('log')

#
# the pseudo-colorbar
#
# the 2nd subplot
ax2 = plt.subplot(gs[1])        
CF.cmap.set_under('yellow')

levls = np.linspace(.001,.01,10)
levls = np.concatenate((levls[:-1],np.linspace(.01,.1,10)))
levls = np.concatenate((levls[:-1],np.linspace(.1,1,10)))
levls = np.concatenate((levls[:-1],np.linspace(1,10,10)))
levls = np.concatenate((levls[:-1],np.linspace(10,100,10)))
levls = np.concatenate((levls[:-1],np.linspace(100,1000,10)))
levls = np.concatenate((levls[:-1],np.linspace(1000,10000,10)))

XC = [np.zeros(len(levls)), np.ones(len(levls))]
YC = [levls, levls]
CM = ax2.contourf(XC,YC,YC, levels=levls, norm = LogNorm(),cmap='YlOrRd')

# log y-scale
ax2.set_yscale('log')  
# y-labels on the right
ax2.yaxis.tick_right()
# no x-ticks
ax2.set_xticks([])
ax2.set_xlabel('$t_{impact}$ \n $(Myr)$',fontsize=9,labelpad=2)
#ax2.xaxis.set_label_position('top') 
CS3 = ax1.contour(time_scales_R.reshape(25,21)/1000.,time_scales_a.reshape(25,21)/R_mars,Z1,
                  norm = LogNorm(),
                  levels = [4.5e3,1e3,1e2,1e1,1],colors='white',linewidths=.25)
plt.clabel(CS3, inline=1, fontsize=7,fmt='  %3.f Myr  ',inline_spacing=3)
plt.show()

/usr/lib/python3/dist-packages/matplotlib/__init__.py:857: UserWarning: text.fontweight is deprecated and replaced with font.weight; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/usr/lib/python3/dist-packages/matplotlib/__init__.py:857: UserWarning: text.fontsize is deprecated and replaced with font.size; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [ ]:
savefig("Fig3a.pdf",dpi=12900)

#py.plot(arr[:,0], arr[:,1], 'o', alpha=0.1, rasterized=True)
#py.savefig('dots.pdf', dpi=400)
